In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

c:\Users\AlbinTouma\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### The purpose of this documnet is to see if an entity is the same in companies and suppliers jsons and or if they are different. 

In [13]:
companies = pd.read_json('iranianentities.json')
suppliers = pd.read_json('iranianSuppliers.json')


In [4]:
is_sanctioned = []
for item in suppliers['Entity']:
    if item in companies['Entity']:
        is_sanctioned.append(True)
    else:
        is_sanctioned.append(False)
        

In [5]:
suppliers['is_sanctioned'] = is_sanctioned
suppliers['is_sanctioned'].value_counts()/len(suppliers)

False    1.0
Name: is_sanctioned, dtype: float64

In [123]:
similarity = []
for item1 in suppliers['Entity']:
    best_match = None
    highest_ratio = 0
    for item2 in companies['Entity']:
        ratio = fuzz.partial_ratio(item1, item2)
        if ratio > highest_ratio and ratio >=95: #When ratio isn't set then best match with similarity ratio of 92 is DES International Co Ltd and MKS International Co. Ltd. 
            highest_ratio = ratio
            best_match = item2
    similarity.append({'Supplier': item1, 'best_match': best_match, 'Similarity Ratio': highest_ratio})
    similarityTable = pd.DataFrame(similarity)
    similarityTable

KeyboardInterrupt: 

In [ ]:
similarityTable = similarityTable.sort_values('Similarity Ratio', ascending=False)
similarityTable.to_csv('similarityRatio.csv')

In [ ]:
similarityTable

,Supplier,best_match,Similarity Ratio
0,Abascience Tech Co. Ltd.,None,0
248,Rick's Manufacturing,None,0
257,Ryu Yon Chol,None,0
256,Ruan Runling,None,0
255,RPP Limited Liability Company,None,0
...,...,...,...
124,Halkbank,None,0
123,Ha Chong Kuk,None,0
122,"Green Wave Telecommunication, Sdn Bhn",None,0
121,Great Wall Airlines,None,0


In [ ]:
similarity2 = []
for item1 in companies['Entity']:
    best_match = None
    highest_ratio = 0
    for item2 in suppliers['Entity']:
        ratio = fuzz.partial_ratio(item1, item2)
        if ratio > highest_ratio and ratio >=95: #When ratio isn't set then best match with similarity ratio of 92 is DES International Co Ltd and MKS International Co. Ltd. 
            highest_ratio = ratio
            best_match = item2
    similarity2.append({'Supplier': item1, 'best_match': best_match, 'Similarity Ratio': highest_ratio})
    similarityTable2 = pd.DataFrame(similarity2)
    

In [ ]:
similarityTable2 = similarityTable2.sort_values('Similarity Ratio', ascending=False)
similarityTable.to_csv('similarityRatio2.csv')


In [6]:
merged = pd.concat([companies, suppliers])
merged['is_sanctioned'] = merged['is_sanctioned'].fillna(True)

merged['unique_id'] = merged.reset_index(drop=False).index + 1
merged = merged[['unique_id', 'Entity', 'Address', 'phone_number', 'company_website', 'Country', 'Weapon Program', 'Suspected Links', 'Information', 'Created', 'Last_updated', 'profile_picture', 'is_sanctioned']]


merged.to_json('combined.json', orient='records', indent=4)

# Duplicates in Data

h1 - Duplicates in data come from the "Featured Suppliers" on the sidebar of the alphabetical list. The scraper includes these
h1 Fails to explain why Iranian Space Agency on p36 of entities isn't a duplicate. Could be a cadence thing if website updates the duplicates change? 

1. Kamran Daneshju is legitimately 2 profiles
2. I think if I reran scraper the new duplicates would be Iranian Space Agency and Materials and Energy Reserach Insttute
    ii. This is the case. Duplicates updated to Iranian Space Agency after scraper on entities run again. 

In [8]:
#There are 11 duplicates
mask = merged.duplicated(subset=['Entity'], keep=False)
duplicates = merged[mask]
duplicates = duplicates.sort_values(ascending=True, by=['Entity'])
duplicates['is_sanctioned'].value_counts() #80 Duplicates in Iranian Entities and 78 duplicates in Supply
duplicates['Entity'].value_counts() #Beijin MicroPilot UAV Flight Contorl Systems, Oje Parvaz, Paravar Pars, ROV appear 39 times each. My suspicion is that the scraper still pulls in sidebar for each serach. 
duplicates['Entity'].value_counts()
duplicate_entities = duplicates[duplicates['is_sanctioned'] == True]
duplicate_suppliers = duplicates[duplicates['is_sanctioned'] == False]
duplicate_suppliers = duplicate_suppliers['Entity'].value_counts()
duplicate_entities = duplicate_entities['Entity'].value_counts()

duplicate = pd.concat([duplicate_entities, duplicate_suppliers], axis=1)
duplicate.columns = ['Entities', 'Suppliers'] 
duplicate

,Entities,Suppliers
Iranian Space Agency,39.0,NaN
Materials and Energy Research Institute,39.0,NaN
Kamran Daneshju,2.0,NaN
Beijing MicroPilot UAV Flight Control Systems,NaN,8.0
ROV Solutions,NaN,7.0
